In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SC
import numpy as np

In [2]:
df = pd.read_csv('Eyes.csv')
X = df[df.columns[1:25]]
Y = df[df.columns[25]]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.10,random_state=42)

sc = SC()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_train, X_test, Y_train, Y_test = np.array(X_train), np.array(X_test), np.array(Y_train), np.array(Y_test)

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation

model = Sequential()
model.add(Dense(4,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [81]:
from scipy.stats import levy

def get_weights(weights):
    #sws = [np.reshape(weights[0:96],(24,4)),np.reshape(weights[96:100],4),np.reshape(weights[100:116],(4,4)),np.reshape(weights[116:120],4),np.reshape(weights[120:124],(4,1)),weights[124]]
    sws = [np.reshape(weights[0:96],(24,4)),[0.,0.,0.,0.],np.reshape(weights[96:112],(4,4)),[0.,0.,0.,0.],np.reshape(weights[112:116],(4,1)),[0.]]
    return sws

r = levy.rvs(size=1)
print(r)
weights = np.random.uniform(-1,1,116)
weights = weights * r
weights = sc.fit_transform(weights.reshape(-1,1))

[1.16866688]


In [91]:
def predict(weights,X_test,Y_test):
    sws = get_weights(weights)
    model.set_weights(sws)
    y_pred = model.predict(X_test)
    # setting a confidence threshhold of 0.9
    y_pred_labels = list(y_pred > 0.9)

    for i in range(len(y_pred_labels)):
        if int(y_pred_labels[i]) == 1 : y_pred_labels[i] = 1
        else : y_pred_labels[i] = 0
    from sklearn.metrics import accuracy_score
    print(accuracy_score(Y_test,y_pred_labels))

predict(weights,X_test,Y_test)

0.68


In [93]:
from algorithms import simulated_annealing,GeomDecay
from neural import NetworkWeights,ContinuousOpt
from activation import sigmoid
clip_max = 5
fitness = NetworkWeights(X_train,Y_train,[24,4,4,1],sigmoid,False,True,0.2)
problem = ContinuousOpt(116,fitness,maximize=False,min_val=-1*clip_max,max_val=clip_max,step=0.2)
print(problem.get_length(),len(weights))
fitted_weights, loss = simulated_annealing(problem,schedule=GeomDecay(),max_attempts=10,max_iters=1000,init_state=weights,curve=False)
predict(fitted_weights,X_test,Y_test)

116 116
0.72
